In [26]:
!pip install gradio

     |████████████████████████████████| 3.6 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 206 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 17.4 MB/s eta 0:00:01          | 901 kB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 140 kB/s  eta 0:00:01
     |████████████████████████████████| 961 kB 14.6 MB/s eta 0:00:01
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4709 sha256=1cc8882e82eb4b7fa051d81d199be36bd4ffe4010adbae101df1d99281374677
  Stored in directory: /home/shariq/.cache/pip/wheels/ff/5b/59/913b443e7369dc04b61f607a746b6f7d83fb65e2e19fcc958d
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=ce901030f2187e141dea1dbd2fcc8efc2e3c59634b51d57a926d045dada5a1fa
  Stored in directory: /home/shariq/.cache/pip/wheels/27/1b/de/34a3e06893944387266e2e7e93d33243589f01ac1b451e367b
Successfully built ffmpy flask-cachebuster


In [27]:
import numpy as np
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer, util
import torch, gc
import gradio as gr
# gc.collect()
# torch.cuda.empty_cache()
# device = torch.device("cpu")

In [3]:
wiki_data = []
with open('tokenized.txt','r',encoding='utf8') as file:
    for line in enumerate(file):
        wiki_data.append(line)

print(len(wiki_data))

5315384


In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
# model = SentenceTransformer('sentence-transformers/paraphrase-albert-small-v2')

In [5]:
# encode corpus to get corpus embeddings

# gc.collect()
# torch.cuda.empty_cache()

data_embeddings = model.encode(wiki_data, show_progress_bar = True)
# data_embeddings = model.encode(wiki_data,batch_size=1,convert_to_tensor=True,show_progress_bar = True)



In [6]:
#Store sentences & embeddings on disc
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'wiki_data': wiki_data, 'data_embeddings': data_embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# #Load corpus and embeddings from disc
with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    wiki_data = stored_data['wiki_data']
    data_embeddings = stored_data['data_embeddings']

print(len(wiki_data))

5315384


In [58]:
def InputQuery(title,abstract,top_k):
    input_query = title+" ||| "+abstract
    input_embeddings = model.encode(input_query)
    cos_scores = util.pytorch_cos_sim(input_embeddings, data_embeddings)[0]
    # Sort the results in decreasing order and get the first top_k
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    output = []
    print("Input Query:", input_query, "\n")
    print("Top", top_k, "most similar topics in the corpus:")
    for i in top_results[0:top_k]:
        output.append(str(wiki_data[i]) + "----->"+ "(Score: %.4f)" % (cos_scores[i])+"\n")
#         print(wiki_data[i], "(Score: %.4f)" % (cos_scores[i]))
#         print('\n')
    output= '\n'.join(output)
    return output

In [23]:
InputQuery(5,"Programming language ","A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declarative form (i.e. the desired result is specified, not how to achieve it).The description of a programming language is usually split into the two components of syntax (form) and semantics (meaning). Some languages are defined by a specification document (for example, the C programming language is specified by an ISO Standard) while other languages (such as Perl) have a dominant implementation that is treated as a reference. Some languages have both, with the basic language defined by a standard and extensions taken from the dominant implementation being common.")

Input Query: Programming language  ||| A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declarati

In [24]:
InputQuery(5,"Motorcycle","This is a sound made by the catA motorcycle, often called a motorbike, bike, or cycle, is a two- or three-wheeled motor vehicle. Motorcycle design varies greatly to suit a range of different purposes: long-distance travel, commuting, cruising, sport (including racing), and off-road riding. Motorcycling is riding a motorcycle and being involved in other related social activity such as joining a motorcycle club and attending motorcycle rallies. The 1885 Daimler Reitwagen made by Gottlieb Daimler and Wilhelm Maybach in Germany was the first internal combustion, petroleum-fueled motorcycle. In 1894, Hildebrand & Wolfmüller became the first series production motorcycle. In 2014, the three top motorcycle producers globally by volume were Honda (28%), Yamaha (17%) (both from Japan), and Hero MotoCorp (India).[6] In developing countries, motorcycles are considered utilitarian due to lower prices and greater fuel economy. Of all the motorcycles in the world, 58% are in the Asia-Pacific and Southern and Eastern Asia regions, excluding car-centric Japan.")

Input Query: Motorcycle ||| This is a sound made by the catA motorcycle, often called a motorbike, bike, or cycle, is a two- or three-wheeled motor vehicle. Motorcycle design varies greatly to suit a range of different purposes: long-distance travel, commuting, cruising, sport (including racing), and off-road riding. Motorcycling is riding a motorcycle and being involved in other related social activity such as joining a motorcycle club and attending motorcycle rallies. The 1885 Daimler Reitwagen made by Gottlieb Daimler and Wilhelm Maybach in Germany was the first internal combustion, petroleum-fueled motorcycle. In 1894, Hildebrand & Wolfmüller became the first series production motorcycle. In 2014, the three top motorcycle producers globally by volume were Honda (28%), Yamaha (17%) (both from Japan), and Hero MotoCorp (India).[6] In developing countries, motorcycles are considered utilitarian due to lower prices and greater fuel economy. Of all the motorcycles in the world, 58% are 

In [59]:
# output_text = for i in range(5):
    
iface = gr.Interface(InputQuery,inputs=["text","text", gr.inputs.Slider(1,5,1)],outputs= "text")
iface.launch(share=True)

Running locally at: http://127.0.0.1:7876/
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://12569.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7876/',
 'https://12569.gradio.app')

Input Query: Programming Language ||| A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.  Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.  Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declar

[2021-10-19 11:44:06,000] ERROR in app: Exception on /api/flag/ [POST]
Traceback (most recent call last):
  File "/home/shariq/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/shariq/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/shariq/anaconda3/lib/python3.8/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/home/shariq/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/shariq/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/shariq/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/sh

Input Query: Programming Language ||| A programming language is a formal language comprising a set of strings that produce various kinds of machine code output. Programming languages are one kind of computer language, and are used in computer programming to implement algorithms.  Most programming languages consist of instructions for computers. There are programmable machines that use a set of specific instructions, rather than general programming languages. Since the early 1800s, programs have been used to direct the behavior of machines such as Jacquard looms, music boxes and player pianos.[1] The programs for these machines (such as a player piano's scrolls) did not produce different behavior in response to different inputs or conditions.  Thousands of different programming languages have been created, and more are being created every year. Many programming languages are written in an imperative form (i.e., as a sequence of operations to perform) while other languages use the declar